# Context Managers and else Blocks


## Do This, Then That: else Blocks Beyond if

- EAFP : Easier to ask for forgiveness than permission.
- LBYL : Look before you leap

---

- the else clause can be used not only in if statements but also in for, while, and try statements.
- The semantics of for/else, while/else, and try/else are closely related, but very different from if/else.

--- 

- for
  - the else block will run only if and when the for `loop runs to completion`
- while
  - the else block will run only if and when the while `loop exits because the condition became falsy`
- try
  - the else block will only run `if no exception is raised` in the try block

---

```py
for item in my_list:
    if item.flavor == 'banana':
        break
else:
    raise ValueError('No banana flavor found')
```

--- 

```py
try:
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()
```


## Context Managers and with Blocks

- Context manager objects exist to control a with statement, just like iterators exist to control a for statement.
- The with statement was designed to simplify the try/finally pattern, which guarantees that some operation is performed after a block of code
- code in the finally clause usually releases a critical resource or restores some previous state that was temporarily changed.

- The context manager protocol consists of the `__enter__` and `__exit__` methods.
- At the start of the with, `__enter__` is invoked on the context manager object.
- The role of the finally clause is played by a call to `__exit__` on the context manager object at the end of the with block

In [4]:
with open('./../README.md') as fp:
    src = src = fp.read()

In [5]:
len(src)

26

In [6]:
fp

<_io.TextIOWrapper name='./../README.md' mode='r' encoding='cp1252'>

In [8]:
print(src)

# Python
Mastering Python



In [9]:
fp.read()

ValueError: I/O operation on closed file.

1. fp is bound to the opened file because the file’s `__enter__` method returns self.
2.  can’t perform I/O with fp because at the end of the with block, the TextIOWrapper.`__exit__` method is called and closes the file.

In [3]:
class LookingGlass:
    
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return "JABBERWOCKY"
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

1. Python invokes `__enter__` with no arguments besides self
2. Hold the original sys.stdout.write method in an instance attribute for later use
3. Monkey patch sys.stdout.write replactin it with our own method
4. Return "JABBERWOCKY" string just so we have something to put in the target variable what
5. Our replacement to sys.stdout.write reverses the text argument and calls the original implementation.
6. Python calls `__exit__` with None, None, None if all went well; if an exception is raised, the three arguments get the exception data, as described next.
7. It’s cheap to import modules again because Python caches them
8. Restore the original method to sys.stdout.write
9. If the exception is not None and itstype is ZeroDivisionError, print a message…
10. …and return True to tell the interpreter that the exception was handled.
11. If `__exit__` returns None or anything but True, any exception raised in the with block will be propagated

In [8]:
manager = LookingGlass()
manager

In [9]:
monster = manager.__enter__()
monster


'JABBERWOCKY'

## The contextlib Utilities

- closing
  - A function to build context managers out of objects that provide a `close()` method but don’t implement the `__enter__`/`__exit__` protocol.
- suppress
  - A context manager to temporarily ignore specified exceptions.
- @contextmanager
  - A decorator that lets you build a context manager from a simple `generator` function, instead of creating a class and implementing the protocol.
- ContextDecorator
  - A base class for defining class-based context managers that can also be used as function decorators, running the entire function within a managed context
- ExitStack
  - A context manager that lets you enter a variable number of context managers. 
  - When the with block ends, ExitStack calls the stacked context managers’ `__exit__` methods in LIFO order (last entered, first exited). 
  - Use this class when you don’t know beforehand how many context managers you need to enter in your with block; for example, when opening all files from an arbitrary list of files at the same time

## Using @contextmanager

- The @contextmanager decorator reduces the boilerplate of creating a context manager
- just implement a generator with a single yield
-  everything before the yield will be executed at the beginning of the while block when the interpreter calls `__enter__`;
- code after yield will run when `__exit__` 

In [11]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text): 
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write

1. Apply the contextmanager decorator
2. Preserve original sys.stdout.write method
3. Define custom reverse_write function; original_write will be available in the closure
4. replace sys.stdout.write with reverse_write
5.  Yield the value that will be bound to the target variable in the as clause of the with statement. This function pauses at this point while the body of the with executes.
6.  When control exits the with block in any way, execution continues after the yield; here the original sys.stdout.write is restored.

Essentially the contextlib.contextmanager decorator wraps the function in a class that implements the `__enter__` and `__exit__` methods. 


The `__enter__` method of that class:
1. Invokes the generator function and holds on to the generator object—let’s call it gen.
2. Calls next(gen) to make it run to the yield keyword.
3. Returns the value yielded by next(gen), so it can be bound to a target variable in the with/as form.


When the with block terminates, the `__exit__` method:
1. Checks an exception was passed as exc_type; if so, gen.throw(exception) is invoked, causing the exception to be raised in the yield line inside the generator function body.
2. Otherwise, next(gen) is called, resuming the execution of the generator function body after the yield

- A serious flaw: if an exception is raised in the body of the with block, the Python interpreter will catch it and raise it again in the yield expression inside looking_glass. 
- But there is no error handling there, so the looking_glass function will abort without ever restoring the original sys.stdout.write method, leaving the system in an invalid state

In [12]:
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    def reverse_write(text):
        original_write(text[::-1])
    sys.stdout.write = reverse_write
    msg = '' 
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError: 
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write 
        if msg:
            print(msg)

1. Create a variable for a possible error message
2. Handle ZeroDivisionError by setting an error message.
3. Undo monkey-patching of sys.stdout.write
4. Display error message, if it was set.

With @contextmanager, the default behavior is inverted: the `__exit__` method provided by the decorator assumes any exception sent into the generator is handled and should be suppressed. You must explicitly re-raise an exception in the decorated function if you don’t want @contextmanager to suppress it.

# Chapter Summary

- discussion of else blocks in for, while, and try statements
- the `__enter__/__exit__` methods,
- with is a tool for factoring out common setup and teardown code, or any pair of operations that need to be done before and after another procedure
- the` @contextmanager` decorator, makes it possible to implement a context manager using a simple generator with one yield